In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

---

In [128]:
path = '/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data'
train = pd.read_csv(path+'/train.csv')
tests = pd.read_csv(path+'/test.csv')
sub = pd.read_csv(path+'/sub.csv')

In [146]:
train.head();train.tail()

,日期,A厂,B厂,year,month,date,sum
1030,2020/10/27,281198.0,241711.0,2020,10,27,522909.0
1031,2020/10/28,275598.0,240281.0,2020,10,28,515879.0
1032,2020/10/29,268544.0,237466.0,2020,10,29,506010.0
1033,2020/10/30,285484.0,237611.0,2020,10,30,523095.0
1034,2020/10/31,275130.0,236154.0,2020,10,31,511284.0


In [147]:
tests.head();tests.tail()

,日期,A厂,B厂,year,month,date,sum
146,2021/03/27,202,941,2021,03,27,1143
147,2021/03/28,853,985,2021,03,28,1838
148,2021/03/29,623,901,2021,03,29,1524
149,2021/03/30,893,692,2021,03,30,1585
150,2021/03/31,572,885,2021,03,31,1457


In [137]:
train['year'], train['month'], train['date'] = 0,0,0
tests['year'], tests['month'], tests['date'] = 0,0,0
train.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in train['日期']])
tests.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in tests['日期']])


In [148]:
train['sum'] = train['A厂'] + train['B厂']
tests['sum'] = tests['A厂'] + tests['B厂']
train['frac_a'] = train['A厂']/train['sum']
tests['frac_a'] = tests['A厂']/tests['sum']
train['frac_b'] = train['B厂']/train['sum']
tests['frac_b'] = tests['B厂']/tests['sum']

In [187]:
train = train.drop('日期', axis = 1)
tests = tests.drop('日期', axis = 1)

In [206]:
dif_col = set(train.columns.to_list()) - set(pd.DataFrame(train.describe()).columns.to_list())

set()

In [205]:
train['year'], train['month'], train['date'] = [train[i].astype(int) for i in list(dif_col)]
tests['year'], tests['month'], tests['date'] = [tests[i].astype(int) for i in list(dif_col)]

---
# baseline

In [357]:
bs_train = train.iloc[:,0:5]
bs_tests = tests.iloc[:,2:5]

In [358]:
bs_tar = bs_train.iloc[:, 0:2]
bs_train = bs_train.iloc[:, 2:]

In [359]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(n_splits = 5, test_size = .2, train_size = .8, random_state = 1026529)

In [360]:
alg1 = LinearRegression()
cv_results_1 = model_selection.cross_validate(alg1, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_1['test_score'].mean()

-0.14551133194229993

In [361]:
alg2 = Ridge()
cv_results_2 = model_selection.cross_validate(alg2, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_2['test_score'].mean()

-0.1455161156470318

In [364]:
bs_mode=alg2.fit(bs_train, y = bs_tar)

In [373]:
sub.iloc[:,1:3] = pd.DataFrame(bs_mode.predict(bs_tests))/100

In [374]:
sub.to_csv('./data/baseline_sub.csv', index=False)

---

## test 2 , the pseudo data

In [3]:
import pandas as pd
import numpy as np
path = '/Users/fyenne/Downloads/booooks/semester5/pythonAAE/open_data_com/data'
train = pd.read_csv(path+'/train.csv')
tests = pd.read_csv(path+'/test.csv')
bssub = pd.read_csv(path+'/baseline_sub.csv')

In [4]:
tests.iloc[:, 1:] = bssub.iloc[:, 1:]

In [5]:
train['year'], train['month'], train['date'] = 0,0,0
tests['year'], tests['month'], tests['date'] = 0,0,0
train.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in train['日期']])
tests.iloc[:,3:6] = pd.DataFrame([i.split('/') for i in tests['日期']])
train['sum'] = train['A厂'] + train['B厂']
tests['sum'] = tests['A厂'] + tests['B厂']
train['frac_a'] = train['A厂']/train['sum']
tests['frac_a'] = tests['A厂']/tests['sum']
train['frac_b'] = train['B厂']/train['sum']
tests['frac_b'] = tests['B厂']/tests['sum']
train = train.drop('日期', axis = 1)
tests = tests.drop('日期', axis = 1)
dif_col = set(train.columns.to_list()) - set(pd.DataFrame(train.describe()).columns.to_list())
train['year'], train['month'], train['date'] = [train[i].astype(int) for i in list(dif_col)]
tests['year'], tests['month'], tests['date'] = [tests[i].astype(int) for i in list(dif_col)]

In [6]:
train.describe()

,A厂,B厂,year,month,date,sum,frac_a,frac_b
count,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000
mean,228833.774879,203170.005797,2018.942029,6.228019,15.731401,432003.780676,0.531211,0.468789
std,36573.680993,38049.912664,0.802873,3.322770,8.805294,61647.423727,0.053596,0.053596
min,96845.000000,74082.000000,2018.000000,1.000000,1.000000,202744.000000,0.341482,0.347467
25%,205118.000000,187928.000000,2018.000000,3.000000,8.000000,407662.000000,0.493630,0.432723
50%,235319.000000,206521.000000,2019.000000,6.000000,16.000000,438152.000000,0.526054,0.473946
75%,256705.500000,229248.500000,2020.000000,9.000000,23.000000,471236.000000,0.567277,0.506370
max,300602.000000,289316.000000,2020.000000,12.000000,31.000000,555221.000000,0.652533,0.658518


In [7]:
tests.describe()

,A厂,B厂,year,month,date,sum,frac_a,frac_b
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,292619.599310,204026.427872,2020.596026,5.841060,15.622517,496646.027182,0.589191,0.410809
std,561.823571,265.593495,0.492325,4.735811,8.780085,827.403917,0.000150,0.000150
min,291861.398665,203666.477793,2020.000000,1.000000,1.000000,495527.876458,0.588991,0.410648
25%,291989.827794,203724.794235,2020.000000,2.000000,8.000000,495714.622028,0.589028,0.410686
50%,292958.595346,204190.537250,2021.000000,3.000000,16.000000,497149.132596,0.589277,0.410723
75%,293087.024475,204248.853691,2021.000000,11.000000,23.000000,497335.878166,0.589314,0.410972
max,293221.330401,204310.547594,2021.000000,12.000000,31.000000,497531.877995,0.589352,0.411009


---

# tensorflow

In [8]:
train_backup = train
train_backup.describe()

,A厂,B厂,year,month,date,sum,frac_a,frac_b
count,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000,1035.000000
mean,228833.774879,203170.005797,2018.942029,6.228019,15.731401,432003.780676,0.531211,0.468789
std,36573.680993,38049.912664,0.802873,3.322770,8.805294,61647.423727,0.053596,0.053596
min,96845.000000,74082.000000,2018.000000,1.000000,1.000000,202744.000000,0.341482,0.347467
25%,205118.000000,187928.000000,2018.000000,3.000000,8.000000,407662.000000,0.493630,0.432723
50%,235319.000000,206521.000000,2019.000000,6.000000,16.000000,438152.000000,0.526054,0.473946
75%,256705.500000,229248.500000,2020.000000,9.000000,23.000000,471236.000000,0.567277,0.506370
max,300602.000000,289316.000000,2020.000000,12.000000,31.000000,555221.000000,0.652533,0.658518


In [9]:
(train).shape

(1035, 8)

In [10]:
target_train = train.iloc[:,0:2]
target_test  = tests.iloc[:,0:2]

In [11]:
tensor_train = train.iloc[:,2:]
tensor_test  = tests.iloc[:,2:]

In [17]:
tensor_train = np.matrix(tensor_train)
target_train = np.matrix(target_train)

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(tensor_train, target_train, test_size=0.25, random_state=7)

In [24]:
# x_train
input_shape =  [8]

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import kerastuner as kt
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=50, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [25]:
def model_builder(hp):
    model = keras.Sequential()
    layers.Dense(1186, activation='relu', input_shape=input_shape),

    hp_units  = hp.Int('units', min_value=593, max_value=1186, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.BatchNormalization()),

    model.add(keras.layers.Dense(units=593, activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(2))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=hp_learning_rate),
                loss=keras.losses.MeanAbsolutePercentageError(),
                metrics=['mean_absolute_percentage_error'])
# tf.keras.losses.MeanAbsoluteError
    return model

In [26]:
tuner = kt.Hyperband(model_builder,
                     objective='mean_absolute_percentage_error',
                     max_epochs=300,
                     factor=4)
                    #  directory='my_dir',
                    #  project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [27]:
tuner.search(x_train, y_train,
             validation_data=(x_valid, y_valid),
             epochs=50, validation_split=0.2,
             batch_size=1024,
             callbacks=[early_stopping])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [1]:
# tensorflow

factory A

In [ ]:
train = train.drop()

In [ ]:
import h2o
h2o.init(nthreads=-1, max_mem_size = "8G")

train_list=list(train.columns)[:-1]
train_1=h2o.H2OFrame(train)
train_1['result']=train_1['A厂']